## Face Recognition

In this demo, we will be training Zumi to recognize faces.

### Step 1: Import Libraries

In [ ]:
print("Importing libraries...")

import sys

sys.path.insert(0, '/home/pi/zumi-python-library')

from util.camera import Camera
from util.screen import Screen

import cv2
import os
import numpy as np
import time
import pickle

from PIL import Image

print("Done!")

### Step 2: Setup

#### Step 2.1 Initialize Variables

In [ ]:
face_detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
recognizer = cv2.face.LBPHFaceRecognizer_create()
font = cv2.FONT_HERSHEY_SIMPLEX
name = None
streaming_image = None
cap = 0

#### Step 2.2 Initialize Camera and Screen

In [ ]:
camera = Camera()
screen = Screen()

#### Step 2.3 Initialize Functions

In [ ]:
def makeLabel():
    labels = 0
    write = False

    if not os.path.exists("./labels.pickle"):
        with open('labels.pickle', 'wb') as labelFile:
            pickle.dump({}, labelFile, protocol=pickle.HIGHEST_PROTOCOL)

    with open('labels.pickle', 'rb') as labelFile:
        labels = pickle.load(labelFile)
        names = list(labels.values())

        if not name in names and name is not None:
            number = len(labels)
            labels[number] = name
            write = True

    if write:
        with open('labels.pickle', 'wb') as labelFile:
            pickle.dump(labels, labelFile, protocol=pickle.HIGHEST_PROTOCOL)

def save_image(image):
    if not os.path.isdir('faces'):
        os.makedirs(os.path.join('faces'))
    file_name = './faces/'

    if name is not None:
        number = 0
        lists = os.listdir('faces')
        for list in lists:
            if name in list:
                number += 1

        file_name = file_name + name + '_' + str(number) + '.jpg'
        cv2.imwrite(file_name, image)

def makeDataset(image):
    global cap
    global streaming_image
    
    makeLabel()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.2, 5)

    for(x, y, w, h) in faces:
        cv2.rectangle(image, (x,y), (x+w, y+h), (255,255,255), 4)
        face_image = gray[y:y+h, x:x+w]
        save_image(face_image)
        cap += 1

        cv2.rectangle(gray, (x,y), (x+w, y+h), (255,255,255), 4)

    streaming_image = cv2.resize(gray, (128, 64))

def getImagesAndLabels(path):
    labels = 0

    with open('labels.pickle', 'rb') as labelFile:
        labels = pickle.load(labelFile)

    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]

    faces = []
    ids = []

    for imagePath in imagePaths:
        PIL_img = Image.open(imagePath).convert('L')
        img_numpy = np.array(PIL_img, 'uint8')

        name = os.path.split(imagePath)[-1].split('_')[0]
        id = -1
        for label in labels:
            if labels[label] == name:
                id = label

        faces.append(img_numpy)
        ids.append(id)

    return faces, ids

def recognition(image):
    global streaming_image

    labels = 0

    with open('labels.pickle', 'rb') as labelFile:
        labels = pickle.load(labelFile)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.2, 5)

    for(x, y, w, h) in faces:
        cv2.rectangle(image, (x,y), (x+w, y+h), (255,255,255), 4)
        Id = recognizer.predict(gray[y:y+h, x:x+w])
        name = labels[Id[0]]
        cv2.putText(image, name, (x, y-5), font, 1, (255,255,255), 3)
        print(name)

        cv2.rectangle(gray, (x,y), (x+w, y+h), (255,255,255), 4)
        cv2.putText(gray, name, (x, y-5), font, 0.5, (255,255,255), 3)

    streaming_image = cv2.resize(gray, (128, 64))

def deleteDataset(name):
    imagePaths = [os.path.join('faces',f) for f in os.listdir('faces')]
    for imagePath in imagePaths:
        imgName = os.path.split(imagePath)[-1].split('_')[0]
        if imgName == name:
            os.remove(imagePath)
    labels = 0
    with open('labels.pickle', 'rb') as labelFile:
        labels = pickle.load(labelFile)
    for key in labels.keys():
        if labels.get(key) == name:
            del labels[key]
            break
    with open('labels.pickle', 'wb') as labelFile:
        pickle.dump(labels, labelFile, protocol=pickle.HIGHEST_PROTOCOL)

### Step 3: Collect Data

* Place Zumi somewhere so that your face is level with her camera.
* Enter a name.
* Have the person whose name you entered look into Zumi's camera.
* Once she sees that person's face, she will take a picture and remember it for later. You can tell she can see a face because a box will be drawn around it on her screen.
* Keep looking into Zumi's camera until the screen says "Done!".

In [ ]:
name = input("Please enter a name: ")

screen.draw_text("Let's take some pictures!")

time.sleep(1)

cap = 0

while True:
    try:
        image = camera.capture()
        makeDataset(image)
        screen.draw_image(Image.fromarray(streaming_image).convert('1'))

        if cap > 50:
            screen.draw_text("Done!")
            break
    except Exception as e:
        print(e)
        camera.close()
        screen.draw_text("")

time.sleep(1)

camera.close()
screen.draw_text("")

### Step 4: Training

Now that Zumi has pictures of faces, she will now try to sort them out.

In [ ]:
print("Now training...")

faces, ids = getImagesAndLabels('faces')
recognizer.train(faces, np.array(ids))
if not os.path.isdir('trainer'):
    os.makedirs(os.path.join('trainer'))
recognizer.save('trainer/trainer.yml')

print("Done!")

### Step 5: Run Model

How did Zumi do? Put a face in front of the camera and Zumi will try to identify it!

In [ ]:
recognizer.read('trainer/trainer.yml')

current_time = time.time()

while(time.time() - current_time < 30):
    try:
        image = camera.capture()
        recognition(image)
        screen.draw_image(Image.fromarray(streaming_image).convert('1'))
    except Exception as e:
        print(e)
        camera.close()
        screen.draw_text("")

### Utilities

These cells aren't necessary for the demo, but they may come in hand.

#### Delete 1 Dataset

In [ ]:
labels = 0
try:
    with open('labels.pickle', 'rb') as labelFile:
        labels = pickle.load(labelFile)
    names = list(labels.values())
    print(names)
    name = input("Please enter a name: ")
    deleteDataset(name)
except:
    print("Error: no dataset")

#### Delete All Datasets

In [ ]:
imagePaths = [os.path.join('faces',f) for f in os.listdir('faces')]
for imagePath in imagePaths:
    os.remove(imagePath)
if os.path.isfile('labels.pickle'):
    os.remove('labels.pickle')
if os.path.isfile('trainer/trainer.yml'):
    os.remove('trainer/trainer.yml')

#### Reset Camera and Screen

In [ ]:
camera.close()
screen.draw_text("")